In [1]:
"""Setup"""
import yaml
import pandas as pd
import pathlib
import uuid
import json
import time
import datetime
from tqdm import tqdm
import humanfriendly
import exifread
import sys
import os
from coco_assistant import COCO_Assistant
from collections import OrderedDict
from jsonmerge import merge
from jsonmerge import Merger

In [4]:
current_path = pathlib.Path(os.path.realpath('../weedcoco/'))

dataset_1_path = current_path / 'deepweeds_to_json/deepweeds_imageinfo.json'
dataset_2_path = current_path / 'cwfid_to_json/cwfid_imageinfo.json'

with open(dataset_1_path) as f:
    dataset_1 = json.load(f, object_pairs_hook=OrderedDict)
    
with open(dataset_2_path) as f:
    dataset_2 = json.load(f, object_pairs_hook=OrderedDict)


In [10]:
# TODO: Determine proper schema for merging 
# TODO: Add valdiation to schema
schema = {
            "properties":{
                "images":{
                    "mergeStrategy":"version"
                },
                "annotations":{
                    "mergeStrategy":"version"
                },
                "categories":{
                    "mergeStrategy":"version"
                },
                "info":{
                    "mergeStrategy":"version"
                },
                "license":{
                    "mergeStrategy":"version"
                },
                "agcontext":{
                    "mergeStrategy":"version"
                },
                "collections":{
                    "mergeStrategy":"version"
                },
                "collection_memberships":{
                    "mergeStrategy":"version"
                }
            }
}

merger = Merger(schema)
result = merger.merge(dataset_1, dataset_2)

BaseInstanceError: 'version' merge strategy: Last entry in the versioned array has no 'value' property. Base not previously generated with this strategy?: #/images/-1

In [5]:


print("Merging annotations")
for j in tqdm(self.jsonfiles):
            with open(os.path.join(self.ann_dir, j)) as a:
                cj = json.load(a)

            ind = self.jsonfiles.index(j)
            # Check if this is the 1st annotation.
            # If it is, continue else modify current annotation
            if ind == 0:
                cann['images'] = cann['images'] + cj['images']
                cann['annotations'] = cann['annotations'] + cj['annotations']
                if 'info' in list(cj.keys()):
                    cann['info'] = cj['info']
                if 'licenses' in list(cj.keys()):
                    cann['licenses'] = cj['licenses']
                cann['categories'] = sorted(cj['categories'], key=lambda i: i['id'])

                last_imid = cann['images'][-1]['id']
                last_annid = cann['annotations'][-1]['id']

                # If last imid or last_annid is a str, convert it to int
                if isinstance(last_imid, str) or isinstance(last_annid, str):
                    logging.debug("String Ids detected. Converting to int")
                    id_dict = {}
                    # Change image id in images field
                    for i, im in enumerate(cann['images']):
                        id_dict[im['id']] = i
                        im['id'] = i

                    # Change annotation id & image id in annotations field
                    for i, im in enumerate(cann['annotations']):
                        im['id'] = i
                        if isinstance(last_imid, str):
                            im['image_id'] = id_dict[im['image_id']]

                last_imid = cann['images'][-1]['id']
                last_annid = cann['annotations'][-1]['id']

            else:

                id_dict = {}
                # Change image id in images field
                for i, im in enumerate(cj['images']):
                    id_dict[im['id']] = last_imid + i + 1
                    im['id'] = last_imid + i + 1

                # Change annotation and image ids in annotations field
                for i, ann in enumerate(cj['annotations']):
                    ann['id'] = last_annid + i + 1
                    ann['image_id'] = id_dict[ann['image_id']]

                # Remap categories
                cmapper = CatRemapper(cann['categories'], cj['categories'])
                cann['categories'], cj['annotations'] = cmapper.remap(cj['annotations'])

                cann['images'] = cann['images'] + cj['images']
                cann['annotations'] = cann['annotations'] + cj['annotations']
                if 'info' in list(cj.keys()):
                    cann['info'] = cj['info']
                if 'licenses' in list(cj.keys()):
                    cann['licenses'] = cj['licenses']

                last_imid = cann['images'][-1]['id']
                last_annid = cann['annotations'][-1]['id']

Merging annotations


AttributeError: 'PosixPath' object has no attribute 'jsonfiles'

In [4]:
with open('merge_test.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

NameError: name 'result' is not defined